# Santander Product Recommendation Data Cleaning

We have shown the EDA of the dataset in previous notebook. To recollect, you can visit the link- 

### Conclusions from EDA for Data Cleaning- 

* In the data, we have a mix of categorical, numerical and ordinal variables.

      We have also seen that a few Variables have different data type in train and test data-

       * age(object - int)
       * antiguedad(object - int)
       * indrel_1mes(object - int)
       * renta(float-object)

* Maximum customers who have their last date as primary customer with the bank(ult_fec_cli_1t value is not NULL) are mostly left out for recommendation. Bank does not wants to recommend them any product. So, we can drop this column.

* Only for a single customer, we had conyuemp=1. We can also drop this column.
* For all the customers, we have tipodom =1. It is not a distinguishing feature. It can also be dropped.

 
* Outliers are present in age, antiguedad and renta. So, they have to be treated by binning. We know that log transformation gave a normal distribution of renta.
 
* The count for ahor, aval, deco, cder is so less that we can exclude them from our products and need not take them into account for recommendations

indrel_1mes and tiprel_1mes were correlated-

* Primary(1) and Co-owner(2) in indrel_1mes implied Active(A) and Inactive(I) customers in tiprel_1mes.
* Former-primary(3) and Former co-owner(4) implied former customers(P) in tiprel_1mes.

If a customer is not a primary customer(indrel=99) at the end of the month, then he starts dropping products(we have observed it for cco and ecue).

These can be used in feature engineering.



## Data Cleaning of Santander Product Recommendation dataset

Now, that we understood the dataset in detail. It is time for us to clean the data.

In this notebook, we are going to focus on two things-
* Missing Values Treatment
* Feature Engineering


### Loading Packages

In [33]:
#importing libraries

import pandas as pd
import numpy as np
import seaborn as sns
sns.set(style="whitegrid")
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook as tqdm
from IPython.display import display
pd.options.display.max_columns = None
import datetime
import warnings                               # To ignore any warnings 
warnings.filterwarnings("ignore")

### Loading Train and Test Data

In [69]:
train=pd.read_csv('train_ver2.csv')           # Loading train and test datasets
test=pd.read_csv('test_ver2.csv')          

In [70]:
train_or=train.copy()                         # making a copy of train and test data.
test_or=test.copy()                          

In [4]:
train.dtypes                                            # gives datatype of each feature in train dataset

fecha_dato                object
ncodpers                   int64
ind_empleado              object
pais_residencia           object
sexo                      object
age                       object
fecha_alta                object
ind_nuevo                float64
antiguedad                object
indrel                   float64
ult_fec_cli_1t            object
indrel_1mes               object
tiprel_1mes               object
indresi                   object
indext                    object
conyuemp                  object
canal_entrada             object
indfall                   object
tipodom                  float64
cod_prov                 float64
nomprov                   object
ind_actividad_cliente    float64
renta                    float64
segmento                  object
ind_ahor_fin_ult1          int64
ind_aval_fin_ult1          int64
ind_cco_fin_ult1           int64
ind_cder_fin_ult1          int64
ind_cno_fin_ult1           int64
ind_ctju_fin_ult1          int64
ind_ctma_f

In [5]:
test.dtypes                                     # gives datatype of each feature in test dataset

fecha_dato                object
ncodpers                   int64
ind_empleado              object
pais_residencia           object
sexo                      object
age                        int64
fecha_alta                object
ind_nuevo                  int64
antiguedad                 int64
indrel                     int64
ult_fec_cli_1t            object
indrel_1mes              float64
tiprel_1mes               object
indresi                   object
indext                    object
conyuemp                  object
canal_entrada             object
indfall                   object
tipodom                    int64
cod_prov                 float64
nomprov                   object
ind_actividad_cliente      int64
renta                     object
segmento                  object
dtype: object

# Data Cleaning

### Missing Values Treatment

In [53]:
train.isnull().sum()

fecha_dato                      0
ncodpers                        0
ind_empleado                27734
pais_residencia             27734
sexo                        27804
age                             0
fecha_alta                  27734
ind_nuevo                   27734
antiguedad                      0
indrel                      27734
ult_fec_cli_1t           13622516
indrel_1mes                149781
tiprel_1mes                149781
indresi                     27734
indext                      27734
conyuemp                 13645501
canal_entrada              186126
indfall                     27734
tipodom                     27735
cod_prov                    93591
nomprov                     93591
ind_actividad_cliente       27734
renta                     2794375
segmento                   189368
ind_ahor_fin_ult1               0
ind_aval_fin_ult1               0
ind_cco_fin_ult1                0
ind_cder_fin_ult1               0
ind_cno_fin_ult1                0
ind_ctju_fin_u

In [4]:
test.isnull().sum()

fecha_dato                    0
ncodpers                      0
ind_empleado                  0
pais_residencia               0
sexo                          5
age                           0
fecha_alta                    0
ind_nuevo                     0
antiguedad                    0
indrel                        0
ult_fec_cli_1t           927932
indrel_1mes                  23
tiprel_1mes                  23
indresi                       0
indext                        0
conyuemp                 929511
canal_entrada              2081
indfall                       0
tipodom                       0
cod_prov                   3996
nomprov                    3996
ind_actividad_cliente         0
renta                         0
segmento                   2248
dtype: int64

* We can see that most of the features in train data have same number of missing values i.e. 27734. But we are not going to drop these missing values as we will loose our data. Instead, a better way is to  impute them.
* Features like conyuemp and ult_fec_cli_1t have almost all the values missing. So, they can be removed from both train and test data.

Missing value variables can be broken down into three categories:

* Missing data for categorical variables: We either impute the mode, as the missing variables are a small subset of the total data, or set to a new 'missing' level if this will imbalance the factor classes.
* Missing data for numerical variables: we can use a more granular imputation, by setting the missing value equal to the average for each province.
* Missing data for product variables: Two products ind_nomina_ult1 and ind_nom_pens_ult1 have missing values. So, we can set their ownership status to 0.

### Imputing train and test data

#### Categorical Variables - Mode Imputation

In [71]:
# For train data

train['ind_empleado'].fillna(train['ind_empleado'].mode()[0], inplace=True) 
train['pais_residencia'].fillna(train['pais_residencia'].mode()[0], inplace=True) 
train['sexo'].fillna(train['sexo'].mode()[0], inplace=True) 

train['ind_nuevo'].fillna(train['ind_nuevo'].mode()[0], inplace=True)
train['indrel'].fillna(train['indrel'].mode()[0], inplace=True)
train['indrel_1mes'].fillna(train['indrel_1mes'].mode()[0], inplace=True) 
train['tiprel_1mes'].fillna(train['tiprel_1mes'].mode()[0], inplace=True) 

train['indresi'].fillna(train['indresi'].mode()[0], inplace=True) 
train['indext'].fillna(train['indext'].mode()[0], inplace=True) 

train['canal_entrada'].fillna(train['canal_entrada'].mode()[0], inplace=True) 
train['indfall'].fillna(train['indfall'].mode()[0], inplace=True)

train['cod_prov'].fillna(train['cod_prov'].mode()[0], inplace=True)         
train['nomprov'].fillna(train['nomprov'].mode()[0], inplace=True)

train['ind_actividad_cliente'].fillna(train['ind_actividad_cliente'].mode()[0], inplace=True)

train['segmento'].fillna(train['segmento'].mode()[0], inplace=True)

# For test data

test['sexo'].fillna(test['sexo'].mode()[0], inplace=True)
test['cod_prov'].fillna(test['cod_prov'].mode()[0], inplace=True)
test['nomprov'].fillna(test['nomprov'].mode()[0], inplace=True)
test.loc[test['indrel_1mes'].isnull(), 'indrel_1mes'] = '1'
test.loc[test['tiprel_1mes'].isnull(), 'tiprel_1mes'] = 'I'
test['segmento'].fillna(test['segmento'].mode()[0], inplace=True)
test['canal_entrada'].fillna(test['canal_entrada'].mode()[0], inplace=True) 

#### Numerical Variables - Imputing mean for each Province

In [72]:
# For train data

train.age = pd.to_numeric(train.age, errors='coerce')                        # Changing their data type from object to int
train.antiguedad = pd.to_numeric(train.antiguedad, errors='coerce')  


train["age"] = train[['age','cod_prov']].groupby("cod_prov").transform(lambda x: x.fillna(x.mean()))                 #Replace age with provincial mean
train["antiguedad"] = train[['antiguedad','cod_prov']].groupby("cod_prov").transform(lambda x: x.fillna(x.mean()))   #Replace antiguedad with provincial mean
train["renta"] = train[['renta','cod_prov']].groupby("cod_prov").transform(lambda x: x.fillna(x.mean()))             #Replace renta with provincial mean


# For test data

test.renta = pd.to_numeric(test.renta, errors='coerce')

test["renta"] = test[['renta','cod_prov']].groupby("cod_prov").transform(lambda x: x.fillna(x.mean()))          #Replace renta with provincial mean

#### Imputing missing values in Products

In [73]:
train.loc[train['ind_nomina_ult1'].isnull(), 'ind_nomina_ult1'] = 0
train.loc[train['ind_nom_pens_ult1'].isnull(), 'ind_nom_pens_ult1'] = 0

#### Dropping the undesired columns

In [74]:
train=train.drop(['ult_fec_cli_1t','conyuemp','tipodom','fecha_alta'],axis=1)
test=test.drop(['ult_fec_cli_1t','conyuemp','tipodom','fecha_alta'],axis=1)

In [8]:
train.isnull().sum()

fecha_dato               0
ncodpers                 0
ind_empleado             0
pais_residencia          0
sexo                     0
age                      0
ind_nuevo                0
antiguedad               0
indrel                   0
indrel_1mes              0
tiprel_1mes              0
indresi                  0
indext                   0
canal_entrada            0
indfall                  0
cod_prov                 0
nomprov                  0
ind_actividad_cliente    0
renta                    0
segmento                 0
ind_ahor_fin_ult1        0
ind_aval_fin_ult1        0
ind_cco_fin_ult1         0
ind_cder_fin_ult1        0
ind_cno_fin_ult1         0
ind_ctju_fin_ult1        0
ind_ctma_fin_ult1        0
ind_ctop_fin_ult1        0
ind_ctpp_fin_ult1        0
ind_deco_fin_ult1        0
ind_deme_fin_ult1        0
ind_dela_fin_ult1        0
ind_ecue_fin_ult1        0
ind_fond_fin_ult1        0
ind_hip_fin_ult1         0
ind_plan_fin_ult1        0
ind_pres_fin_ult1        0
i

In [9]:
test.isnull().sum()

fecha_dato               0
ncodpers                 0
ind_empleado             0
pais_residencia          0
sexo                     0
age                      0
ind_nuevo                0
antiguedad               0
indrel                   0
indrel_1mes              0
tiprel_1mes              0
indresi                  0
indext                   0
canal_entrada            0
indfall                  0
cod_prov                 0
nomprov                  0
ind_actividad_cliente    0
renta                    0
segmento                 0
dtype: int64

Now, we have imputed all the missing values and removed all the undesired columns from both train and test data.

Now, the next step is to treat the outliers present in numerical variables. We will do this by creating bins. We will create bins in the same way as we had done in the EDA.

### Outlier Treatment

#### Age

In [150]:
train.age.describe()

count    1.364731e+07
mean     4.019258e+01
std      1.716938e+01
min      2.000000e+00
25%      2.400000e+01
50%      3.900000e+01
75%      5.000000e+01
max      1.640000e+02
Name: age, dtype: float64

In [75]:
bins=[0,19,60,200] 
group=['teen','working','old']  

train['age']=pd.cut(train['age'],bins,labels=group)
test['age']=pd.cut(test['age'],bins,labels=group)

#### Antiguedad

In [41]:
train.antiguedad.describe()

count    1.364731e+07
mean     7.663323e+01
std      1.670107e+03
min     -9.999990e+05
25%      2.300000e+01
50%      5.000000e+01
75%      1.350000e+02
max      2.560000e+02
Name: antiguedad, dtype: float64

In [76]:
train.antiguedad[train.antiguedad<0] =0                                   # Removing -999999
test.antiguedad[test.antiguedad<0] =0

In [77]:
bins=[-1,17,66,500] 
group=['new','medium','senior']  

train['antiguedad']=pd.cut(train['antiguedad'],bins,labels=group)
test['antiguedad']=pd.cut(test['antiguedad'],bins,labels=group)

#### Renta

In [78]:
train["renta"] = np.log(train["renta"])
test["renta"] = np.log(test["renta"])

In [45]:
train.renta.describe()

count    1.364731e+07
mean     1.158806e+01
std      5.848313e-01
min      7.092349e+00
25%      1.123011e+01
50%      1.156680e+01
75%      1.198557e+01
max      1.717916e+01
Name: renta, dtype: float64

In [79]:
bins = [0,10,12,19]
group=['poor','medium','rich']

train['renta'] = pd.cut(train.renta,bins=bins,labels=group)
test['renta'] = pd.cut(test.renta,bins=bins,labels=group)

#### Indrel_1mes

In [80]:
train.indrel_1mes=train.indrel_1mes.replace('P',0)                    # potential customer = 0
train.indrel_1mes=pd.to_numeric(train['indrel_1mes'],errors='coerce')

test.indrel_1mes=test.indrel_1mes.replace('P',0)                      # potential customer = 0
test.indrel_1mes=pd.to_numeric(test['indrel_1mes'],errors='coerce')

#### Pais_residenica

In [81]:
li=['ES']

er=train[train['pais_residencia'].isin(li)]
q=train[~train['pais_residencia'].isin(li)]

er1=test[test['pais_residencia'].isin(li)]
q1=test[~test['pais_residencia'].isin(li)]


q['pais_residencia']='others'
q1['pais_residencia']='others'

train=er.append(q)
test=er1.append(q1)

#### Canal_entrada

In [83]:
li=['KAT','KHE','KFC']

er=train[train['canal_entrada'].isin(li)]
q=train[~train['canal_entrada'].isin(li)]

er1=test[test['canal_entrada'].isin(li)]
q1=test[~test['canal_entrada'].isin(li)]


q['canal_entrada']='others'
q1['canal_entrada']='others'

train=er.append(q)
test=er1.append(q1)

In [84]:
train.shape, test.shape

((13647309, 44), (929615, 20))

#### Nomprov

In [85]:
li=['MADRID',' BARCELONA','VALENCIA','SEVILLA']

er=train[train['nomprov'].isin(li)]
q=train[~train['nomprov'].isin(li)]

er1=test[test['nomprov'].isin(li)]
q1=test[~test['nomprov'].isin(li)]


q['nomprov']='others'
q1['nomprov']='others'

train=er.append(q)
test=er1.append(q1)

#### Removing the least popular products

In [89]:
train=train.drop(['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cder_fin_ult1','ind_deco_fin_ult1'],axis=1)

#### Downloading the cleaned files

In [90]:
train.to_csv('train_final.csv')

test.to_csv('test_final.csv')

We will be using these files for Model Building.